### Recordatorio: Hacer push del notebook de DinoGame después de entrenar y graficar métricas. Luego copiar esta celda y ejecutarla, si sirve, volver a hacer push.

In [ ]:
import shap
import torch
import random

# Instanciar el entorno
env = DinoGame()

# 1. Cargar el modelo guardado
state_size = 12
action_size = 3
agent = DQNAgent(state_size, action_size)
agent.model.load_state_dict(torch.load('best_dino_model.pth'))
agent.model.eval()  # Poner el modelo en modo evaluación

# 2. Crear un conjunto de datos representativo de estados
# Simulando algunos estados aleatorios del entorno DinoGame
sample_states = []
for _ in range(100):  # Obtener 100 ejemplos de estados
    state = env.reset()
    done = False
    while not done:
        action = random.randint(0, action_size - 1)
        next_state, _, done = env.step(action)
        sample_states.append(state)
        state = next_state
sample_states = torch.FloatTensor(sample_states[:100])  # Convertir a tensor y limitar a 100 estados

# 3. Configurar SHAP DeepExplainer con el modelo cargado
explainer = shap.GradientExplainer(agent.model, sample_states)

# 4. Calcular los valores SHAP
shap_values = explainer.shap_values(sample_states)

# 5. Reorganizar los valores SHAP por acción
# shap_values es una lista de matrices (una por estado); cada matriz tiene dimensiones (12, 3).
# Extraer cada columna para agrupar por acción.
shap_values_by_action = [np.array([state[:, i] for state in shap_values]) for i in range(action_size)]

# 6. Visualizar los valores SHAP para cada acción
feature_names = [f'Feature_{i}' for i in range(state_size)]
for i in range(action_size):
    print(f"Valores SHAP para la acción {i}")
    shap.summary_plot(shap_values_by_action[i], sample_states.numpy(), feature_names=feature_names)
